In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sp

from physical_constants import *
from eos_functions import *

##############################################################################################################################

coupling=[0.3]
valorgamma=[0.5]
valordelta=[0.1]

for valor1,delta in enumerate ( valordelta ):
    
    for valor2,gamma in enumerate (valorgamma ):
        
        for valor3,g_0 in enumerate ( coupling ):
            
            
            
            outputfile="delta{}_{}gamma{}.dat".format(delta,g_0,gamma)
            outputfile1="delta{}_xp_{}gamma{}.dat".format(delta,g_0,gamma)
            
            
            #####################      
            
            
            e0=15.8     
            s0=32. 
            g=g_0*800 # Unitats de MeV·fm^3
            
            
            
            num_densities=1000 

            n0=0.16 

            numden_i=0.2*n0 
            numden_f=10*n0


            numden=np.linspace( numden_i,numden_f,num_densities) 
            
            
            ### Newton raphson ###
            
            precisio=1e-12 

            xp=np.zeros(num_densities)+1e-6 
        

            for i,n in enumerate( numden ): 
                xpold=xp[i] 
    
    
                S3=np.power(4.*s0*np.power( n/n0,gamma )-g*n,3) 
                A=1./(3.*np.power(pi,2)*n*hbc3)
    
                def fun(xpold): 
                    fun=A*S3*(np.power(1-2*xpold,3)/xpold) - 1
                    return fun
    
                def der(xpold):    
                    der=A*S3*(-6*xpold*np.power(-2*xpold+1,2) - np.power(-2*xpold+1,3))/np.power(xpold,2)
                    return der
        
      
                xp[i]=sp.newton(fun,xpold,fprime=der,rtol=precisio) 
    
                if xp[i]<0:
                    print('xp negatius!')
            
            
            ### End Newton raphson ###
            
            
            
            
            
            
            # Primer calculem la densitat de neutrons n_n i de protons n_p (en fm^-3)
            n_p=numden*xp
            n_n=numden*(1.-xp)

            # Ara calculem els paràmetres principals pels electrons
            electron_eden_units=np.power(m_el,4)/hbc3/3./np.power(pi,2)

            mu_e=4*s0*np.power(numden/n0,gamma)*(1.-2.*xp)+g*xp*numden # potencial químic de l'electró
            pf_e=hbc*np.power((3*np.power(pi,2)*xp*numden),1./3.)
            n_e=n_p
        
            
            
            # Vector amb les energies per partícula per cada valor de n
            e_binding=(e0/n0)*numden*((numden-n0*(2.+delta))/(n0+delta*numden)) \
            + s0*np.power((numden/n0),gamma)*np.power((1.-2.*xp),2.)
            e_coupling=0.5*g*xp*(1-xp)*numden #

            # I ara podem trobar la densitat d'energia dels nucleons, hi ho posem en un vector en MeV fm-3
            edens_n=np.zeros(num_densities)
            edens_n=e_binding*numden + e_coupling*numden + n_n*mneut + n_p*mprot
            
            
            
            derivada1=-e0*(n0-numden)*np.power(numden,2)\
            *( (2.+delta)*n0 + delta*numden )/( n0*np.power( n0+delta*numden,2 ) )
            derivada2=s0*gamma*np.power(n0,-gamma)*np.power( (n_n - n_p),2 )*np.power(numden,gamma-1.)
            derivada3=0.5*g*xp*(1.-xp)*np.power(numden,2) # part deguda al coupling


            derivada=derivada1+derivada2+derivada3

            p_nuc=derivada
            
            
            # Ara calculem la densitat d'energia en MeV*fm-3
            edens_e=electron_eden_units*energy(pf_e/m_el) 
            edens_e=edens_e + 0.5*g*xp*(1.-xp)*np.power(numden,2) 
            
            
            derivada1_el=electron_eden_units*pressure(pf_e/m_el)
            derivada2_el=0.5*g*xp*(1.-xp)*np.power(numden,2) # part deguda al coupling


            derivada_el=derivada1_el+derivada2_el

            p_el=derivada_el
            
            
            
            edens=edens_n + edens_e
            press=p_nuc + p_el
            
            u=numden/n0
            #Ho escrivim en un fitxer
            data_to_write = np.array( [edens[:],press[:],u[:]] ).T
            with open(outputfile,"w+") as file_id :
                np.savetxt(file_id,data_to_write,
                           fmt=["%16.6E","%16.6E","%16.6E"],header="  eden [MeVfm-3] Pres [MeVfm-3]     Den/0.16")

            data_to_write1 = np.array( [xp[:],u[:]] ).T
            with open(outputfile1,"w+") as file_id :
                np.savetxt(file_id,data_to_write1,fmt=["%16.6E","%16.6E"],header="  xp   Den/0.16")
            
            

## Espai

In [2]:
for valor1,delta in enumerate ( valordelta ):
    
    for valor2,gamma in enumerate (valorgamma ):
        
        for valor3,g_0 in enumerate ( coupling ):
            
            
            eden_crust=np.loadtxt("crust_bo.dat",usecols=0)
            pres_crust=np.loadtxt("crust_bo.dat",usecols=1)
            den_crust=np.loadtxt("crust_bo.dat",usecols=2)
            pres_core=np.loadtxt("delta{}_{}gamma{}.dat".format(delta,g_0,gamma),usecols=1)
            den_core=np.loadtxt("delta{}_{}gamma{}.dat".format(delta,g_0,gamma),usecols=2)
            
            
            outputfile="delta{}_{}gamma{}.dat".format(delta,g_0,gamma)
            outputfile1="delta{}_xp_{}gamma{}.dat".format(delta,g_0,gamma)
            outputfile2="delta{}_CRUST_{}gamma{}.dat".format(delta,g_0,gamma)
            
            
            pres_crust_list=pres_crust.tolist()
            den_crust_list=den_crust.tolist()
            pres_core_list=pres_core.tolist()
            den_core_list=den_core.tolist()
            
            len_crust=len(pres_crust_list)
            len_core=len(pres_core_list)
            
            
            index_core=len_core
            
            exit=0
            
            for no_importa in range(len_core):
                if exit > 5:
                    break
                else:
                    index_core=index_core-1
                for i in range(len_crust):
                    if pres_crust_list[i] > pres_core_list[index_core] and den_crust_list[i] < den_core_list[index_core]:
                        exit=exit+10
                        break
            
            
            edens_crust=np.zeros(i+1)
            press_crust=np.zeros(i+1)
            u_crust=np.zeros(i+1)
            for compte in range(i+1):
                edens_crust[compte]=eden_crust[compte]
                press_crust[compte]=pres_crust[compte]
                u_crust[compte]=den_crust[compte]
                
            
            
            
            #####################      
            
            
            e0=15.8     
            s0=32. 
            g=g_0*800 # Unitats de MeV·fm^3
            
            
            
            num_densities=1000 

            n0=0.16 

            numden_i=den_core[index_core]*n0 
            numden_f=10*n0


            numden=np.linspace( numden_i,numden_f,num_densities) 
            
            
            ### Newton raphson ###
            
            precisio=1e-12 

            xp=np.zeros(num_densities)+1e-6 
        

            for i,n in enumerate( numden ): 
                xpold=xp[i] 
    
    
                S3=np.power(4.*s0*np.power( n/n0,gamma )-g*n,3) 
                A=1./(3.*np.power(pi,2)*n*hbc3)
    
                def fun(xpold): 
                    fun=A*S3*(np.power(1-2*xpold,3)/xpold) - 1
                    return fun
    
                def der(xpold):    
                    der=A*S3*(-6*xpold*np.power(-2*xpold+1,2) - np.power(-2*xpold+1,3))/np.power(xpold,2)
                    return der
        
      
                xp[i]=sp.newton(fun,xpold,fprime=der,rtol=precisio) 
    
                if xp[i]<0:
                    print('xp negatius!')
            
            
            ### End Newton raphson ###
            
            
            
            
            
            
            # Primer calculem la densitat de neutrons n_n i de protons n_p (en fm^-3)
            n_p=numden*xp
            n_n=numden*(1.-xp)

            # Ara calculem els paràmetres principals pels electrons
            electron_eden_units=np.power(m_el,4)/hbc3/3./np.power(pi,2)

            mu_e=4*s0*np.power(numden/n0,gamma)*(1.-2.*xp)+g*xp*numden # potencial químic de l'electró
            pf_e=hbc*np.power((3*np.power(pi,2)*xp*numden),1./3.)
            n_e=n_p
        
            
            
            # Vector amb les energies per partícula per cada valor de n
            e_binding=(e0/n0)*numden*((numden-n0*(2.+delta))/(n0+delta*numden)) \
            + s0*np.power((numden/n0),gamma)*np.power((1.-2.*xp),2.)
            e_coupling=0.5*g*xp*(1-xp)*numden #

            # I ara podem trobar la densitat d'energia dels nucleons, hi ho posem en un vector en MeV fm-3
            edens_n=np.zeros(num_densities)
            edens_n=e_binding*numden + e_coupling*numden + n_n*mneut + n_p*mprot
            
            
            
            derivada1=-e0*(n0-numden)*np.power(numden,2)\
            *( (2.+delta)*n0 + delta*numden )/( n0*np.power( n0+delta*numden,2 ) )
            derivada2=s0*gamma*np.power(n0,-gamma)*np.power( (n_n - n_p),2 )*np.power(numden,gamma-1.)
            derivada3=0.5*g*xp*(1.-xp)*np.power(numden,2) # part deguda al coupling


            derivada=derivada1+derivada2+derivada3

            p_nuc=derivada
            
            
            # Ara calculem la densitat d'energia en MeV*fm-3
            edens_e=electron_eden_units*energy(pf_e/m_el) 
            edens_e=edens_e + 0.5*g*xp*(1.-xp)*np.power(numden,2) 
            
            
            derivada1_el=electron_eden_units*pressure(pf_e/m_el)
            derivada2_el=0.5*g*xp*(1.-xp)*np.power(numden,2) # part deguda al coupling


            derivada_el=derivada1_el+derivada2_el

            p_el=derivada_el
            
            
            
            edens=edens_n + edens_e
            press=p_nuc + p_el
            
            u=numden/n0
            #Ho escrivim en un fitxer
            data_to_write = np.array( [edens[:],press[:],u[:]] ).T
            with open(outputfile,"w+") as file_id :
                np.savetxt(file_id,data_to_write,
                           fmt=["%16.6E","%16.6E","%16.6E"],header="  eden [MeVfm-3] Pres [MeVfm-3]     Den/0.16")

            data_to_write1 = np.array( [xp[:],u[:]] ).T
            with open(outputfile1,"w+") as file_id :
                np.savetxt(file_id,data_to_write1,fmt=["%16.6E","%16.6E"],header="  xp   Den/0.16")
            
            data_to_write2 = np.array( [edens_crust[:],press_crust[:],u_crust[:]] ).T
            with open(outputfile2,"w+") as file_id :
                np.savetxt(file_id,data_to_write2,
                           fmt=["%16.6E","%16.6E","%16.6E"],header="  eden [MeVfm-3] Pres [MeVfm-3]     Den/0.16")
            